<a href="https://colab.research.google.com/github/kjindal19/ennovate-challenge/blob/main/Second_Approach_(Using_Neural_Networks)_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is data Augmentation 

In [2]:
from google.colab import drive
from google.colab.patches import cv2_imshow
from keras.preprocessing.image import ImageDataGenerator
import glob
import cv2
import numpy as np
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
# Define data augmentation parameters
datagen = ImageDataGenerator(
    rotation_range=40, # rotate image by 10 degrees
    zoom_range=0.1, # zoom in/out by 10%
    width_shift_range=0.1, # shift image horizontally by 10%
    height_shift_range=0.1, # shift image vertically by 10%
    brightness_range=[0.7, 1.3], # adjust brightness by random factor between 0.7 and 1.3
)
training_images = []

# Load reference image and training images
reference_image = cv2.imread(r'/content/drive/MyDrive/Colab Notebooks/Ennovate Coding Challenge/reference.png')
for file in glob.glob(r'/content/drive/MyDrive/Colab Notebooks/Ennovate Coding Challenge/Train/*.JPG'):
    training_images.append(cv2.imread(file))



In [11]:

# Perform data augmentation on each training image and save it to disk
for i, image in enumerate(training_images):
    # Expand image dimensions to match batch size of 1
    image = np.expand_dims(image, axis=0)
    
    # Generate augmented images and save to disk
    j = 0
    for batch in datagen.flow(image, batch_size=1, save_to_dir=r'/content/drive/MyDrive/Colab Notebooks/Ennovate Coding Challenge/augmented_data', save_prefix=f'train1{i}', save_format='jpg'):
        j += 1
        if j == 10: # generate 10 augmented images per original image
            break


THis is model training

In [3]:
# Importing the necessary libraries
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers




In [ ]:
# Loading the reference image and the training images
ref_image = cv2.imread(r'/content/drive/MyDrive/Colab Notebooks/Ennovate Coding Challenge/reference.png')
train_images = []
for file in glob.glob(r'/content/drive/MyDrive/Colab Notebooks/Ennovate Coding Challenge/Train/*.JPG'):
    scale_percent = 20
    src = cv2.imread(file)
    width = int(src.shape[1] * scale_percent / 100)
    height = int(src.shape[0] * scale_percent / 100)
    dsize = (width, height)
    train_images.append(cv2.resize(src, dsize))

for file in glob.glob(r'/content/drive/MyDrive/Colab Notebooks/Ennovate Coding Challenge/Train/*.jpg'):
    scale_percent = 20
    src = cv2.imread(file)
    width = int(src.shape[1] * scale_percent / 100)
    height = int(src.shape[0] * scale_percent / 100)
    dsize = (width, height)
    train_images.append(cv2.resize(src, dsize))

# Creating the input and output pairs for training the model
# The input is the noisy image and the output is the reference image
X_train = np.array(train_images)
y_train = np.array([ref_image for _ in range(1000)])

# Normalizing the pixel values to [-1, 1] range
X_train = X_train / 127.5 - 1.0
y_train = y_train / 127.5 - 1.0


<ipython-input-4-eef8e0e0ccad>:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train = np.array(train_images)


In [ ]:

# Defining the neural network model
# The model is a convolutional autoencoder that learns to reconstruct the input image
model = keras.Sequential()
model.add(layers.Input(shape=(None, None, 3)))
model.add(layers.Conv2D(64, (3, 3), activation="relu", padding="same"))
model.add(layers.MaxPooling2D((2, 2), padding="same"))
model.add(layers.Conv2D(32, (3, 3), activation="relu", padding="same"))
model.add(layers.MaxPooling2D((2, 2), padding="same"))
model.add(layers.Conv2D(16, (3, 3), activation="relu", padding="same"))
model.add(layers.MaxPooling2D((2, 2), padding="same"))
model.add(layers.Conv2D(16, (3, 3), activation="relu", padding="same"))
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Reshape((1, 1, 16)))
model.add(layers.UpSampling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation="relu", padding="same"))
model.add(layers.UpSampling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation="relu", padding="same"))
model.add(layers.UpSampling2D((2, 2)))
model.add(layers.Conv2D(3, (3, 3), activation="tanh", padding="same"))

# Compiling the model with mean squared error loss and Adam optimizer
model.compile(loss="mse", optimizer="adam")


In [ ]:

# Training the model for 50 epochs with a batch size of 32
model.fit(X_train, y_train, epochs=50, batch_size=32)

# Saving the model on the disk
model.save("denoise_model.h5")


This is model using

In [ ]:

# Loading the model from the disk
model = keras.models.load_model("denoise_model.h5")


# Loading the input image to be processed
input_image = cv2.imread("input_image.jpg")

# Normalizing the pixel values to [-1, 1] range
input_image = input_image / 127.5 - 1.0

# Reshaping the input image to match the model input shape
input_image = input_image.reshape(1, *input_image.shape)

# Predicting the output image using the model
output_image = model.predict(input_image)

# Reshaping the output image to match the original image shape
output_image = output_image.reshape(*output_image.shape[1:])

# Denormalizing the pixel values to [0, 255] range
output_image = (output_image + 1.0) * 127.5

# Converting the output image to uint8 type
output_image = output_image.astype(np.uint8)

# Saving the output image on the disk
cv2.imwrite("output_image.jpg", output_image)